In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

In [2]:
# task='mortality'
# days = '90days'

task='readmission'
days = '15days'

modality_order = [
    'Tabular', 'Lab', 'Note',
    'Tabular Only', 'Tab + Lab', 'Tab + Note', 
    'Multimodal', 'Total'
]


df = pd.read_csv(Path('Results') / f"Whole_{task.capitalize()}_{days}.csv")
df['Modality'] = ['Tabular', 'Lab', 'Note', 'Multimodal', 'Tabular Only', 'Tab + Lab', 'Tab + Note', 'Total']
ours_scores = df.set_index('Modality').loc[modality_order].reset_index()
ours_scores = ours_scores[['Modality', 'ROC-AUC Mean', 'PR-AUC Mean', 'N']].rename(columns = {'ROC-AUC Mean' : 'AUROC', 'PR-AUC Mean' : 'AUPRC'})
ours_scores

,Modality,AUROC,AUPRC,N
0,Tabular,67.50,43.21,46018
1,Lab,59.97,32.02,36801
2,Note,63.24,34.77,27242
3,Tabular Only,76.77,59.60,7550
4,Tab + Lab,68.99,45.03,11224
5,Tab + Note,65.81,34.37,1665
6,Multimodal,65.26,37.32,25577
7,Total,68.72,44.00,46018


In [3]:
def load_keys(adm_ids_path) : 
    adm_ids = pickle.load(open(adm_ids_path, 'rb'))
    code_ids, lab_ids, note_ids = adm_ids['test_code_ids'], adm_ids['test_lab_ids'], adm_ids['test_discharge_ids']
    code_ids = np.array(code_ids).astype(int)
    lab_ids = np.array(lab_ids).astype(int)
    note_ids = np.array(note_ids).astype(int)
    return set(code_ids), set(lab_ids), set(note_ids)

def get_score(df ,ids) : 
    target_df = df.query('pid in @ids')
    labels = target_df['label'].values
    probs = target_df['score'].values
    roc_auc = roc_auc_score(labels, probs)
    precision, recall, _ = precision_recall_curve(labels, probs)
    pr_auc = auc(recall, precision)
    return roc_auc, pr_auc, len(labels)

In [4]:
muse_dir = Path('previous_study/MUSE/Results/MUSE/output')
data_dir = Path('/home/data/2025_MIMICIV_processed/mimic4/')

suffix = '-finetune'
output_dirs = list(muse_dir.glob(f'*{task}*{suffix}'))

total_output = []
for output_dir in output_dirs : 
    prediction_path = output_dir / 'predictions.txt'
    prediction = pd.read_csv(prediction_path, header=None)
    prediction.columns = ['pid', 'label', 'score']
    
    seed = int(prediction_path.parent.stem.split('-')[-2])
    
    adm_ids_path = data_dir / f"task:{task}_{days}/admission_ids_seed_{seed}.pkl"
    adm_ids = pickle.load(open(adm_ids_path, 'rb'))
    code_ids, lab_ids, note_ids = load_keys(adm_ids_path)
    
    # ## ['Tab All', 'Lab All', 'Note All', 'Tab Only', 'Tab + Lab', 'Tab + Note', 'Tab + Lab + Note']
    tab_all_keys = np.array(list(code_ids))
    total_output.append([seed, 'Tabular', *get_score(prediction, tab_all_keys)])
    lab_all_keys = np.array(list(lab_ids))
    total_output.append([seed, 'Lab', *get_score(prediction, lab_all_keys)])
    note_all_keys = np.array(list(note_ids))
    total_output.append([seed, 'Note', *get_score(prediction, note_all_keys)])
    
    tab_only_keys = np.array(list( # Tabular에만 있고 나머지는 차집합
        code_ids - lab_ids - note_ids
    ))
    total_output.append([seed, 'Tabular Only', *get_score(prediction, tab_only_keys)])
    
    
    tab_lab_keys = np.array(list( # Tab + Lab - Note
        lab_ids - note_ids
    ))
    total_output.append([seed, 'Tab + Lab', *get_score(prediction, tab_lab_keys)])
    
    tab_note_keys = np.array(list( # Tab + Note - Lab
        note_ids - lab_ids
    ))
    total_output.append([seed, 'Tab + Note', *get_score(prediction, tab_note_keys)])
    
    mm_keys = np.array(list(
        code_ids & lab_ids & note_ids
    ))
    total_output.append([seed, 'Multimodal', *get_score(prediction, mm_keys)])
    
    all_keys = np.array(list(
        code_ids | lab_ids | note_ids
    ))
    total_output.append([seed, 'Total', *get_score(prediction, all_keys)])
    
total_df = pd.DataFrame(total_output, columns = ['seed', 'Modality', 'AUROC', 'AUPRC', 'N'])
total_df_avg = total_df.groupby(['Modality']).mean().reset_index().drop(columns = ['seed'])
total_df_avg['N'] = total_df_avg['N'].astype(int)
total_df_avg[['AUROC', 'AUPRC']] = total_df_avg[['AUROC', 'AUPRC']].apply(lambda x : round(x*100, 2))

muse_scores = total_df_avg.set_index('Modality').loc[modality_order].reset_index()
muse_scores

,Modality,AUROC,AUPRC,N
0,Tabular,66.70,40.66,46018
1,Lab,64.04,36.19,36801
2,Note,62.99,33.91,27242
3,Tabular Only,75.07,55.25,7550
4,Tab + Lab,66.94,41.00,11224
5,Tab + Note,63.22,31.24,1665
6,Multimodal,62.87,34.04,25577
7,Total,66.70,40.66,46018


In [5]:
muse_scores

,Modality,AUROC,AUPRC,N
0,Tabular,66.70,40.66,46018
1,Lab,64.04,36.19,36801
2,Note,62.99,33.91,27242
3,Tabular Only,75.07,55.25,7550
4,Tab + Lab,66.94,41.00,11224
5,Tab + Note,63.22,31.24,1665
6,Multimodal,62.87,34.04,25577
7,Total,66.70,40.66,46018


In [6]:
ours_scores

,Modality,AUROC,AUPRC,N
0,Tabular,67.50,43.21,46018
1,Lab,59.97,32.02,36801
2,Note,63.24,34.77,27242
3,Tabular Only,76.77,59.60,7550
4,Tab + Lab,68.99,45.03,11224
5,Tab + Note,65.81,34.37,1665
6,Multimodal,65.26,37.32,25577
7,Total,68.72,44.00,46018


In [8]:
score = 'AUROC'
merged = pd.merge(
    ours_scores[['Modality', score]],
    muse_scores[['Modality', score]],
    on='Modality',
    suffixes=('_ours', '_muse')
)
merged[f'Delta {score} (ours - muse)'] = merged[f'{score}_ours'] - merged[f'{score}_muse']
merged = merged.set_index('Modality').loc[modality_order].reset_index()
merged

,Modality,AUROC_ours,AUROC_muse,Delta AUROC (ours - muse)
0,Tabular,67.50,66.70,0.80
1,Lab,59.97,64.04,-4.07
2,Note,63.24,62.99,0.25
3,Tabular Only,76.77,75.07,1.70
4,Tab + Lab,68.99,66.94,2.05
5,Tab + Note,65.81,63.22,2.59
6,Multimodal,65.26,62.87,2.39
7,Total,68.72,66.70,2.02
